# NLP with oil and renewable energies project

#### Get libraries

In [1]:
import requests
import re
from bs4 import BeautifulSoup
from collections import Counter
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB  
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler, NearMiss
import numpy as np
from PIL import Image
from sklearn.linear_model import LogisticRegression
import argparse
from matplotlib import pyplot as plt
import wordcloud
import requests, json
import os
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import seaborn as sns

In [2]:
corpus_oil_price=pd.read_csv("data/oil_price.csv", index_col=0)
corpus_oil_price["date_published"] = pd.to_datetime(corpus_oil_price["date_published"], errors='coerce', utc=True).dt.to_period('m')
corpus_oil_price

C:\Users\kaypa\anaconda3\lib\site-packages\pandas\core\arrays\datetimes.py:1088: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(


,title,date_published,content
0,$70 Oil Won't Keep OPEC Happy,2021-11,One day&mdash;that was all it took oil prices ...
1,Why This Oil Price Slump May Not Be Bad News F...,2021-12,Crude oil prices are plunging under the weight...
2,The Double Irony Of Biden's Plea To OPEC,2021-11,As I wrote in February of this year in&nbsp;Th...
3,The Oil Price Rally Is Far From Over,2021-11,How much higher could oil prices go This is th...
4,"Biden Threatens OPEC With Undisclosed ""Tools""",2021-11,"""There are other tools in the arsenal that we ..."
...,...,...,...
2183,A Disobliging Update on the Price of Oil,2010-01,There still seems to be a deplorable uncertain...
2184,What Happens When the Wells Run Dry,2010-01,One nagging question that the industrial world...
2185,Petro Unfriendly California Changing it's Stan...,2009-12,The financial crisis changed the way people th...
2186,New Techniques Oil Companies are Using in Dril...,2009-11,As the politics and philosophical arguments ab...


In [3]:
a = corpus_oil_price.groupby('date_published')[['title']].count()
a['date'] = a.index
a.to_csv("data/oilprice.csv")

In [4]:
corpus_alternative_energy=pd.read_csv("data/alternative_energy.csv", index_col=0)
corpus_alternative_energy["date_published"] = pd.to_datetime(corpus_alternative_energy["date_published"], errors='coerce', utc=True).dt.to_period('m')

C:\Users\kaypa\anaconda3\lib\site-packages\pandas\core\arrays\datetimes.py:1088: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(


In [5]:
b = corpus_alternative_energy.groupby('date_published')[['title']].count()
b['date'] = b.index
b.to_csv("data/alternative.csv")
b

,title,date
date_published,,
2009-07,6,2009-07
2009-09,2,2009-09
2009-10,2,2009-10
2009-11,2,2009-11
2010-02,5,2010-02
...,...,...
2021-07,12,2021-07
2021-08,13,2021-08
2021-09,7,2021-09


In [6]:
corpus_crude_oil=pd.read_csv("data/crude_oil_df.csv", index_col=0)
corpus_crude_oil["date_published"] = pd.to_datetime(corpus_crude_oil["date_published"], errors='coerce', utc=True).dt.to_period('m')

C:\Users\kaypa\anaconda3\lib\site-packages\pandas\core\arrays\datetimes.py:1088: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(


In [7]:
c = corpus_crude_oil.groupby('date_published')[['title']].count()
c['date'] = c.index
c.to_csv("data/crudeoil.csv")

In [8]:
corpus_oil_price2 = pd.read_csv('data/corpus_oil_price_3.csv')
corpus_alternative_energies2 = pd.read_csv('data/corpus_alternative_energies_3.csv')
corpus_crude_oil2 = pd.read_csv('data/corpus_crude_oil_3.csv')
corpus_crude_oil2

,Unnamed: 0,Unnamed: 0.1,title,date_published,content,baseline_sentiment,vader_sentiment,most_common_words
0,0,0,Oil Bulls Rejoice As Biden's Supply Strategy B...,2021-11-24T18:00:00-06:00,President Bidenrsquos threat to oil producers ...,-1,-0.9926,"['oil', 'prices', 'OPEC', 'Bidenrsquos', 'rele..."
1,1,1,"OPEC To Add 400,000 Bpd In January Despite Oil...",2021-12-02T08:53:55-06:00,The OPEC group is sticking to its plan to ease...,0,0.4497,"[' ', 'OPEC', 'bpd', 'January', '400,000', 'me..."
2,2,2,UAE Pumps $6 Billion Into Oil And Gas Expansio...,2021-11-24T17:00:00-06:00,The U.S.-sponsored lsquorelationship normalisa...,1,0.9875,"['oil', 'UAErsquos', 'billion', 'UAE', 'US$', ..."
3,3,3,China Keeps Markets In The Dark About SPR Release,2021-11-24T16:00:00-06:00,The volume of the expected Chinese release of ...,-1,-0.9552,"['oil', ' ', 'release', 'crude', 'reserves', '..."
4,4,4,Oil Markets Unimpressed By Small Crude Invento...,2021-11-24T09:36:00-06:00,"Crude oil inventories rose last week, while ga...",-1,-0.9601,"['million', ' ', 'barrels', 'week', 'bpd', 'cr..."
...,...,...,...,...,...,...,...,...
5000,5000,5000,A Detailed Guide on the Many Different Types o...,2009-12-02T17:12:01-06:00,Some people arbitrarily speak about oil as if ...,-1,-0.9962,"['oil', 'crude', 'oils', 'WTI', 'Oils', 'Brent..."
5001,5001,5001,What is Peak Oil Theory A Thorough Look at Thi...,2009-10-21T16:17:44-05:00,Currently there is a lot of debate going on re...,1,-0.1567,"['oil', 'time', 'world', 'supply', 'production..."
5002,5002,5002,Oil Shale - So Just What Is It,2009-09-24T17:26:55-05:00,People often say ldquoYou canrsquot squeeze bl...,-1,0.9816,"['oil', 'shale', 'energy', 'process', 'rock', ..."
5003,5003,5003,Oil is not a Villain Here's what it's done for us,2009-09-21T16:45:02-05:00,"For decades now, oil has often been vilified a...",1,0.9987,"['oil', 'energy', 'world', 'time', 'Fact', 'ye..."


In [9]:
corpus_oil_price2['most_common_words'] = corpus_oil_price2['most_common_words'].str.replace("[","")
corpus_oil_price2['most_common_words'] = corpus_oil_price2['most_common_words'].str.replace("]","")
corpus_oil_price2['most_common_words'] = corpus_oil_price2['most_common_words'].str.replace("'","")
corpus_oil_price2['most_common_words'] = corpus_oil_price2['most_common_words'].str.replace("',","")
corpus_oil_price2['most_common_words'] = corpus_oil_price2['most_common_words'].str.replace(",'","")
corpus_oil_price2['most_common_words'] = corpus_oil_price2['most_common_words'].str.replace(",","")

In [10]:
col_one_list_2 = corpus_oil_price2['most_common_words'].tolist()
col_one_list = []
for i in col_one_list_2:
    listRes = list(i.split(" "))
    col_one_list.append(listRes)   
col_one_list

[['oil',
  '',
  '',
  'OPEC',
  '$',
  'release',
  'new',
  'bpd',
  'million',
  'barrels',
  'rdquo',
  'prices'],
 ['',
  '',
  'production',
  'shale',
  'oil',
  'prices',
  'approach',
  'industry',
  'drillers',
  'way',
  'plunging',
  'latest'],
 ['oil',
  '',
  '',
  'Biden',
  'prices',
  'Administration',
  'production',
  'mdash',
  'U.S.',
  '',
  '',
  '',
  '',
  'year',
  'fossil'],
 ['oil',
  '',
  '',
  'prices',
  'demand',
  '$',
  'higher',
  'barrel',
  'year',
  'said',
  'supply',
  'bullish'],
 ['',
  '',
  'OPEC',
  'oil',
  'pump',
  'tools',
  'President',
  'Biden',
  'said',
  'crude',
  'Washington',
  'global'],
 ['OPEC',
  'Morse',
  'production',
  '',
  '',
  'oil',
  'prices',
  'said',
  'U.S.',
  'capacity',
  'year',
  'continue'],
 ['',
  '',
  'oil',
  'million',
  'prices',
  'U.S.',
  'Fed',
  '$',
  'inflation',
  'supply',
  'crude',
  'barrels'],
 ['',
  '',
  'crude',
  'oil',
  'barrels',
  'prices',
  'market',
  'Fed',
  'inventory',

In [19]:
stopwords = ["$","d","oil","prices","price",""," ", "year", "week", "barrel", "million", "percent", "said", "bpd", 
             "barrels",
            "oil", "crude", "ndash", "rdquo", "day", "Crude", "Oil"
            
            
            ]
for c in col_one_list:
        #for i in range(len(c)):
        #    c[i] = c[i].lower()
        for word in c:  # iterating on a copy since removing will mess things up
            if word in stopwords:
                c.remove(word)
col_one_list

[['OPEC', 'release', 'new'],
 ['production',
  'shale',
  'approach',
  'industry',
  'drillers',
  'way',
  'plunging',
  'latest'],
 ['Biden', 'Administration', 'production', 'mdash', 'U.S.', 'fossil'],
 ['demand', 'higher', 'supply', 'bullish'],
 ['OPEC', 'pump', 'tools', 'President', 'Biden', 'Washington', 'global'],
 ['OPEC', 'Morse', 'production', 'U.S.', 'capacity', 'continue'],
 ['U.S.', 'Fed', 'inflation', 'supply'],
 ['market', 'Fed', 'inventory', 'API'],
 ['energy', 'inflation', 'supply', 'demand', 'high', 'major'],
 ['production', 'OPEC', '100', 'U.S.', 'WTI', 'years'],
 ['China', 'reserve', 'Chinese', 'supply', 'news', 'gasoline'],
 ['gas', 'natural', 'inventories', 'demand', 'w', 'energy', 'years'],
 ['weather', 'cold', 'gas', '90'],
 ['China',
  'Monday',
  'COVID',
  'flare',
  'market',
  'quickly',
  'Chinese',
  'demand'],
 ['Thursday', 'Wednesday'],
 ['nbsp', 'minister', '100', 'global', 'OPEC'],
 ['coal', 'early', 'Wednesday', 'intervention', 'China'],
 ['2021', 'S

In [20]:
stopwords = ["$","d","oil","prices","price",""," ", "year", "week", "barrel", "million", "percent", "said", "bpd", 
             "barrels",
            "oil", "crude", "ndash", "rdquo", "day", "Crude", "Oil"]
for c in col_one_list:
        #for i in range(len(c)):
        #    c[i] = c[i].lower()
        for word in c:  # iterating on a copy since removing will mess things up
            if word in stopwords:
                c.remove(word)
col_one_list

[['OPEC', 'release', 'new'],
 ['production',
  'shale',
  'approach',
  'industry',
  'drillers',
  'way',
  'plunging',
  'latest'],
 ['Biden', 'Administration', 'production', 'mdash', 'U.S.', 'fossil'],
 ['demand', 'higher', 'supply', 'bullish'],
 ['OPEC', 'pump', 'tools', 'President', 'Biden', 'Washington', 'global'],
 ['OPEC', 'Morse', 'production', 'U.S.', 'capacity', 'continue'],
 ['U.S.', 'Fed', 'inflation', 'supply'],
 ['market', 'Fed', 'inventory', 'API'],
 ['energy', 'inflation', 'supply', 'demand', 'high', 'major'],
 ['production', 'OPEC', '100', 'U.S.', 'WTI', 'years'],
 ['China', 'reserve', 'Chinese', 'supply', 'news', 'gasoline'],
 ['gas', 'natural', 'inventories', 'demand', 'w', 'energy', 'years'],
 ['weather', 'cold', 'gas', '90'],
 ['China',
  'Monday',
  'COVID',
  'flare',
  'market',
  'quickly',
  'Chinese',
  'demand'],
 ['Thursday', 'Wednesday'],
 ['nbsp', 'minister', '100', 'global', 'OPEC'],
 ['coal', 'early', 'Wednesday', 'intervention', 'China'],
 ['2021', 'S

In [21]:
word_cnt = {}
for words in col_one_list:
    for word in words:
        if word not in word_cnt:
            word_cnt[word] = 1
        else:
            word_cnt[word] += 1
    
word_cnt_df_oil_price = pd.DataFrame({'word': [k for k in word_cnt.keys()], 'cnt': [v for v in word_cnt.values()]})
word_cnt_df_oil_price.sort_values(by='cnt', ascending=False, inplace=True)
word_cnt_df_oil_price = word_cnt_df_oil_price.nlargest(25, 'cnt')
word_cnt_df_oil_price.to_csv('data/word_cnt_df_oil_price.csv')

In [22]:
corpus_alternative_energies2['most_common_words'] = corpus_alternative_energies2['most_common_words'].str.replace("[","")
corpus_alternative_energies2['most_common_words'] = corpus_alternative_energies2['most_common_words'].str.replace("]","")
corpus_alternative_energies2['most_common_words'] = corpus_alternative_energies2['most_common_words'].str.replace("'","")
corpus_alternative_energies2['most_common_words'] = corpus_alternative_energies2['most_common_words'].str.replace("',","")
corpus_alternative_energies2['most_common_words'] = corpus_alternative_energies2['most_common_words'].str.replace(",'","")
corpus_alternative_energies2['most_common_words'] = corpus_alternative_energies2['most_common_words'].str.replace(",","")

In [23]:
col_one_list_2 = corpus_alternative_energies2['most_common_words'].tolist()
col_one_list = []
for i in col_one_list_2:
    listRes = list(i.split(" "))
    col_one_list.append(listRes)   
col_one_list

[['',
  '',
  'energy',
  'tons',
  'metals',
  '$',
  'costs',
  'battery',
  'prices',
  'renewable',
  'copper',
  'big'],
 ['',
  '',
  'report',
  'Southeast',
  'Asiarsquos',
  'carbon',
  'green',
  'climate',
  'Hardcastle',
  'action',
  '$',
  'Net'],
 ['energy',
  'Europe',
  '',
  '',
  'prices',
  'gas',
  'European',
  'transition',
  'high',
  'said',
  'EU',
  'plans'],
 ['energy',
  '',
  '',
  'global',
  'like',
  'transition',
  'renewable',
  'clean',
  'fossil',
  'fuels',
  'world',
  'oil'],
 ['',
  '',
  'carbon',
  'solar',
  'oil',
  'Chevron',
  'wind',
  'energy',
  'power',
  'gas',
  'renewable',
  'emissions'],
 ['solar',
  '',
  '',
  'capacity',
  'energy',
  'installations',
  'GW',
  'study',
  'Energy',
  'cost',
  'U.S.',
  'government'],
 ['Manchin',
  '',
  '',
  'Sen.',
  'Democratic',
  '$',
  '3.5',
  'trillion',
  'plan',
  'said',
  'CNN',
  'bill'],
 ['oil',
  'major',
  '',
  '',
  'energy',
  'green',
  'climate',
  'hydrocarbon',
  'proj

In [31]:
stopwords = ["$","d","oil","prices","price",""," ", "year", "week", "barrel", "million", "percent", "said", "bpd", 
             "barrels",
            "oil", "crude", "ndash", "rdquo", "day", "Crude", "Oil", "Energy", "billion", "energy", "power"           
            
            ]
for c in col_one_list:
        #for i in range(len(c)):
        #    c[i] = c[i].lower()
        for word in c:  # iterating on a copy since removing will mess things up
            if word in stopwords:
                c.remove(word)
col_one_list

[['tons', 'metals', 'costs', 'battery', 'renewable', 'copper', 'big'],
 ['report',
  'Southeast',
  'Asiarsquos',
  'carbon',
  'green',
  'climate',
  'Hardcastle',
  'action',
  'Net'],
 ['Europe', 'gas', 'European', 'transition', 'high', 'EU', 'plans'],
 ['global',
  'like',
  'transition',
  'renewable',
  'clean',
  'fossil',
  'fuels',
  'world'],
 ['carbon', 'solar', 'Chevron', 'wind', 'gas', 'renewable', 'emissions'],
 ['solar',
  'capacity',
  'installations',
  'GW',
  'study',
  'cost',
  'U.S.',
  'government'],
 ['Manchin', 'Sen.', 'Democratic', '3.5', 'trillion', 'plan', 'CNN', 'bill'],
 ['major', 'green', 'climate', 'hydrocarbon', 'projects', 'UAE', 'world'],
 ['solar', 'capacity', 'silver', 'supply', '2021', 'years', 'GW', 'Rystad'],
 ['Saudi', 'plans', 'Arabia', 'production', 'market', 'Kingdom'],
 ['wind', 'potential', 'United', 'global', 'capacity', 'world', 'States'],
 ['consumption',
  'renewable',
  '2020',
  'category',
  'global',
  'rate',
  'growth',
  'decade

In [32]:
stopwords = ["$","d","oil","prices","price",""," ", "year", "week", "barrel", "million", "percent", "said", "bpd", 
             "barrels",
            "oil", "crude", "ndash", "rdquo", "day", "Crude", "Oil", "Energy", "billion", "energy", "power"           
            
            ]
for c in col_one_list:
        #for i in range(len(c)):
        #    c[i] = c[i].lower()
        for word in c:  # iterating on a copy since removing will mess things up
            if word in stopwords:
                c.remove(word)
col_one_list

[['tons', 'metals', 'costs', 'battery', 'renewable', 'copper', 'big'],
 ['report',
  'Southeast',
  'Asiarsquos',
  'carbon',
  'green',
  'climate',
  'Hardcastle',
  'action',
  'Net'],
 ['Europe', 'gas', 'European', 'transition', 'high', 'EU', 'plans'],
 ['global',
  'like',
  'transition',
  'renewable',
  'clean',
  'fossil',
  'fuels',
  'world'],
 ['carbon', 'solar', 'Chevron', 'wind', 'gas', 'renewable', 'emissions'],
 ['solar',
  'capacity',
  'installations',
  'GW',
  'study',
  'cost',
  'U.S.',
  'government'],
 ['Manchin', 'Sen.', 'Democratic', '3.5', 'trillion', 'plan', 'CNN', 'bill'],
 ['major', 'green', 'climate', 'hydrocarbon', 'projects', 'UAE', 'world'],
 ['solar', 'capacity', 'silver', 'supply', '2021', 'years', 'GW', 'Rystad'],
 ['Saudi', 'plans', 'Arabia', 'production', 'market', 'Kingdom'],
 ['wind', 'potential', 'United', 'global', 'capacity', 'world', 'States'],
 ['consumption',
  'renewable',
  '2020',
  'category',
  'global',
  'rate',
  'growth',
  'decade

In [33]:
word_cnt = {}
for words in col_one_list:
    for word in words:
        if word not in word_cnt:
            word_cnt[word] = 1
        else:
            word_cnt[word] += 1
    
word_cnt_df_renewables = pd.DataFrame({'word': [k for k in word_cnt.keys()], 'cnt': [v for v in word_cnt.values()]})
word_cnt_df_renewables.sort_values(by='cnt', ascending=False, inplace=True)
word_cnt_df_renewables = word_cnt_df_renewables.nlargest(25, 'cnt')
word_cnt_df_renewables.to_csv('data/word_cnt_df_renewables.csv')

In [34]:
corpus_crude_oil2['most_common_words'] = corpus_crude_oil2['most_common_words'].str.replace("[","")
corpus_crude_oil2['most_common_words'] = corpus_crude_oil2['most_common_words'].str.replace("]","")
corpus_crude_oil2['most_common_words'] = corpus_crude_oil2['most_common_words'].str.replace("'","")
corpus_crude_oil2['most_common_words'] = corpus_crude_oil2['most_common_words'].str.replace("',","")
corpus_crude_oil2['most_common_words'] = corpus_crude_oil2['most_common_words'].str.replace(",'","")
corpus_crude_oil2['most_common_words'] = corpus_crude_oil2['most_common_words'].str.replace(",","")

In [35]:
col_one_list_2 = corpus_crude_oil2['most_common_words'].tolist()
col_one_list = []
for i in col_one_list_2:
    listRes = list(i.split(" "))
    col_one_list.append(listRes)   
col_one_list

[['oil',
  'prices',
  'OPEC',
  'Bidenrsquos',
  'release',
  'crude',
  'markets',
  'producers',
  'plan',
  'SPR',
  'market'],
 ['',
  '',
  'OPEC',
  'bpd',
  'January',
  '400000',
  'meeting',
  'production',
  'expected',
  'group',
  'cuts',
  'policy'],
 ['oil',
  'UAErsquos',
  'billion',
  'UAE',
  'US$',
  'key',
  'million',
  'deal',
  'Abu',
  'Dhabi',
  'ADNOC'],
 ['oil',
  '',
  '',
  'release',
  'crude',
  'reserves',
  'China',
  'prices',
  'said',
  'Chinese',
  'releasing',
  'coordinated'],
 ['million',
  '',
  '',
  'barrels',
  'week',
  'bpd',
  'crude',
  'draw',
  '$',
  'oil',
  'inventories',
  'previous'],
 ['release',
  'oil',
  '',
  '',
  'reserve',
  'barrels',
  'OPEC',
  'crude',
  'according',
  'million',
  'prices',
  'strategic'],
 ['oil',
  'ban',
  'crude',
  'exports',
  '',
  '',
  'House',
  'U.S.',
  'prices',
  'White',
  'Khanna',
  'potential'],
 ['',
  '',
  'oil',
  'release',
  'million',
  'barrels',
  'SPR',
  'coordinated',
  '

In [41]:
stopwords = ["$","d","oil","prices","price",""," ", "year", "week", "barrel", "million", "percent", "said", "bpd", 
             "barrels",
            "oil", "crude", "ndash", "rdquo", "day", "Crude", "Oil", "Energy", "billion", "energy", "power",      
            "null", "oil", "barrels", "$", "million", "bpd", "crude", "years"
            ]
for c in col_one_list:
        #for i in range(len(c)):
        #    c[i] = c[i].lower()
        for word in c:  # iterating on a copy since removing will mess things up
            if word in stopwords:
                c.remove(word)
col_one_list

[['OPEC',
  'Bidenrsquos',
  'release',
  'markets',
  'producers',
  'plan',
  'SPR',
  'market'],
 ['OPEC',
  'January',
  '400000',
  'meeting',
  'production',
  'expected',
  'group',
  'cuts',
  'policy'],
 ['UAErsquos', 'UAE', 'US$', 'key', 'deal', 'Abu', 'Dhabi', 'ADNOC'],
 ['release', 'reserves', 'China', 'Chinese', 'releasing', 'coordinated'],
 ['draw', 'inventories', 'previous'],
 ['release', 'reserve', 'OPEC', 'according', 'strategic'],
 ['ban', 'exports', 'House', 'U.S.', 'White', 'Khanna', 'potential'],
 ['release', 'SPR', 'coordinated', 'OPEC', 'Tuesday'],
 ['quarter',
  'peer',
  'shale',
  '2021',
  'group',
  'reinvestment',
  'yearrsquos',
  'debt'],
 ['release', 'reserves', 'Japan', 'India', 'consumers', 'United', 'States'],
 ['gas', 'pump', 'average', 'production', 'Biden', 'U.S.'],
 ['gas', 'SPR', 'release', 'China', 'India', 'USD'],
 ['stocks', 'OECD', 'build', 'stock', 'IEA'],
 ['China', 'release', 'Reuters', 'Biden', 'strategic', 'National', 'Food'],
 ['Iraq', 

In [42]:
stopwords = ["$","d","oil","prices","price",""," ", "year", "week", "barrel", "million", "percent", "said", "bpd", 
             "barrels",
            "oil", "crude", "ndash", "rdquo", "day", "Crude", "Oil", "Energy", "billion", "energy", "power",      
            "null", "oil", "barrels", "$", "million", "bpd", "crude", "years"
            ]
for c in col_one_list:
        #for i in range(len(c)):
        #    c[i] = c[i].lower()
        for word in c:  # iterating on a copy since removing will mess things up
            if word in stopwords:
                c.remove(word)
col_one_list

[['OPEC',
  'Bidenrsquos',
  'release',
  'markets',
  'producers',
  'plan',
  'SPR',
  'market'],
 ['OPEC',
  'January',
  '400000',
  'meeting',
  'production',
  'expected',
  'group',
  'cuts',
  'policy'],
 ['UAErsquos', 'UAE', 'US$', 'key', 'deal', 'Abu', 'Dhabi', 'ADNOC'],
 ['release', 'reserves', 'China', 'Chinese', 'releasing', 'coordinated'],
 ['draw', 'inventories', 'previous'],
 ['release', 'reserve', 'OPEC', 'according', 'strategic'],
 ['ban', 'exports', 'House', 'U.S.', 'White', 'Khanna', 'potential'],
 ['release', 'SPR', 'coordinated', 'OPEC', 'Tuesday'],
 ['quarter',
  'peer',
  'shale',
  '2021',
  'group',
  'reinvestment',
  'yearrsquos',
  'debt'],
 ['release', 'reserves', 'Japan', 'India', 'consumers', 'United', 'States'],
 ['gas', 'pump', 'average', 'production', 'Biden', 'U.S.'],
 ['gas', 'SPR', 'release', 'China', 'India', 'USD'],
 ['stocks', 'OECD', 'build', 'stock', 'IEA'],
 ['China', 'release', 'Reuters', 'Biden', 'strategic', 'National', 'Food'],
 ['Iraq', 

In [43]:
word_cnt = {}
for words in col_one_list:
    for word in words:
        if word not in word_cnt:
            word_cnt[word] = 1
        else:
            word_cnt[word] += 1
    
word_cnt_df_crude = pd.DataFrame({'word': [k for k in word_cnt.keys()], 'cnt': [v for v in word_cnt.values()]})
word_cnt_df_crude.sort_values(by='cnt', ascending=False, inplace=True)
word_cnt_df_crude = word_cnt_df_crude.nlargest(25, 'cnt')
word_cnt_df_crude.to_csv('data/word_cnt_df_crude.csv')